In [401]:


import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline



In [402]:
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [403]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [404]:
import random
random.seed(42)
random.shuffle(words)

In [405]:
block_size = 8 # context length from how man y words we predict the next one

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

xtr,  ytr  = build_dataset(words[:n1])     # 80%
xdev, ydev = build_dataset(words[n1:n2])   # 10%
xte,  yte  = build_dataset(words[n2:])     #10%

torch.Size([182580, 8]) torch.Size([182580])
torch.Size([22767, 8]) torch.Size([22767])
torch.Size([22799, 8]) torch.Size([22799])


In [406]:
for x,y in zip(xtr[:20], ytr[:20]):
  print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> e
.......e --> b
......eb --> r
.....ebr --> i
....ebri --> m
...ebrim --> a
..ebrima --> .
........ --> h
.......h --> i
......hi --> l
.....hil --> t
....hilt --> o
...hilto --> n
..hilton --> .
........ --> j
.......j --> h
......jh --> e
.....jhe --> n
....jhen --> e
...jhene --> .


In [407]:
class Linear:
  
  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 #kaiming init
    self.bias = torch.zeros(fan_out) if bias else None
  
  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])

#-------------------------------------------------------------------------
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # parameters (trained using back propagation they are learnable)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # buffers (trained with a running momentum update)
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)
  
  def __call__(self, x):
    #forward pass
    if self.training:
      if x.ndim==2:
        dim=0
      elif x.ndim==3:
        dim=(0,1) 
      xmean = x.mean(dim, keepdim=True) # batch mean
      xvar = x.var(dim, keepdim=True) # batch variance
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    # update the buffers
    if self.training:
      with torch.no_grad(): #they are not going to compute gradients
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]
#------------------------------------------------------------------
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return []
  
#--------------------------------------------------------------------------------
class Embedding:
  def __init__(self,num_embeddings,embedding_dim):
    self.weight=torch.randn(num_embeddings,embedding_dim)
  def __call__(self,ix):
    self.out=self.weight[ix]
    return self.out
  def parameters(self):
    return [self.weight]
  
#-----------------------------------------------------------------------------------
class FlattenConsecutive:
  def __init__(self,n):
    self.n=n
  def __call__(self,x):
    B,T,C=x.shape
    x=x.view(B,T//self.n,C*self.n)
    if x.shape[1]==1:
      x=x.squeeze(1)
    self.out=x
    return self.out
  def parameters(self):
    return []
  
#----------------------------------------------------------------------------------------
class Sequential: #based on the concept of containers(way of organizing layers)
  def __init__(self, layers):
    self.layers = layers
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    self.out = x
    return self.out
  
  def parameters(self):
    # get parameters of all layers and stretch them out into one list
    return [p for layer in self.layers for p in layer.parameters()]
  

In [408]:
torch.manual_seed(42);

In [ ]:
n_embd = 24 #  dimension of the character embedding vectors C
n_hidden = 128 # the number of neurons in the hidden layer of the MLP

model = Sequential([
  Embedding(vocab_size,n_embd),#implemented c[xtr[ix]] inside the layers
  FlattenConsecutive(2),Linear(n_embd * 2, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2),Linear(n_hidden*2,n_hidden ,bias=False), BatchNorm1d(n_hidden), Tanh(),
  FlattenConsecutive(2),Linear(n_hidden * 2,n_hidden ,bias=False), BatchNorm1d(n_hidden), Tanh(),
  Linear(n_hidden, vocab_size),
])


with torch.no_grad():
  # last layer: make less confident
  layers[-1].weight *= 0.1 

parameters =  model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True #default value is false

76579


In [410]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, xtr.shape[0], (batch_size,))
  xb, yb = xtr[ix], ytr[ix] # batch X,Y
  
  # forward pass
  logits=model(xb)
  loss = F.cross_entropy(logits, yb) # loss function
  
  # backward pass 
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # updating
  lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track status
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())


      0/ 200000: 3.5778
  10000/ 200000: 2.5472
  20000/ 200000: 2.1711
  30000/ 200000: 2.6860
  40000/ 200000: 1.8854
  50000/ 200000: 2.4277
  60000/ 200000: 2.1162
  70000/ 200000: 1.8266
  80000/ 200000: 2.2092
  90000/ 200000: 1.9398
 100000/ 200000: 2.1306
 110000/ 200000: 2.0065
 120000/ 200000: 1.7098
 130000/ 200000: 2.0440
 140000/ 200000: 1.6520
 150000/ 200000: 2.1748
 160000/ 200000: 1.8280
 170000/ 200000: 2.1274
 180000/ 200000: 2.4208
 190000/ 200000: 1.6216


In [411]:
# plt.plot(torch.tensor(lossi).view(-1, 1000).mean(1))


In [412]:
for layer in model.layers:
  print(layer.__class__.__name__,':',tuple(layer.out.shape))

Embedding : (32, 8, 24)
FlattenConsecutive : (32, 4, 48)
Linear : (32, 4, 128)
BatchNorm1d : (32, 4, 128)
Tanh : (32, 4, 128)
FlattenConsecutive : (32, 2, 256)
Linear : (32, 2, 128)
BatchNorm1d : (32, 2, 128)
Tanh : (32, 2, 128)
FlattenConsecutive : (32, 256)
Linear : (32, 128)
BatchNorm1d : (32, 128)
Tanh : (32, 128)
Linear : (32, 27)


In [413]:
for layer in model.layers:
  layer.training = False

In [414]:
# e=torch.randn(32,4,68)
# emean=e.mean(0,keepdim=True)
# evar=e.var(0,keepdim=True)
# ehat=(e-emean)/torch.sqrt(evar + 1e-5)
# ehat.shape

In [415]:
torch.no_grad() #this disables gradient tracking
def split_loss(split):
    x,y={ 
    'train' : (xtr,ytr),
    'val' : (xdev,ydev),
    'test': (xte,yte),
    }[split] # it takes the train and val string in the list
    logits=model(x) 
    loss=F.cross_entropy(logits,y)# loss function
    print(split,loss.item())
split_loss('train')
split_loss('val')

train 1.766515851020813
val 1.985236644744873


In [416]:
for x,y in zip(xtr[7:14],ytr[7:14]):
    print(''.join(itos[ix.item()] for ix in x),'-->',itos[y.item()])

........ --> h
.......h --> i
......hi --> l
.....hil --> t
....hilt --> o
...hilto --> n
..hilton --> .


In [417]:
logits=model(xtr[[7]])
logits.shape

torch.Size([1, 27])

In [431]:
o = []
for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      logits=model(torch.tensor([context]))
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1).item()
      # shift the context window and track the samples
      context = context[1:] + [ix]
      
      # if we sample the special '.' token, break
      if ix == 0:
        break
      out.append(ix)
    print(''.join(itos[i] for i in out)) 
    o.append(''.join(itos[i] for i in out))

adeliza
brenkon
jesslyn
gwendalina
farruq
ahmail
korine
ceraldo
arlani
matleigh
lionna
anaziah
aryaan
mikaimar
christopher
lucas
lyrice
india
saels
annal


In [432]:
c =0 
for i in o:
    if i in words:
        c+=1
    else:
        print(i)
c

brenkon
gwendalina
farruq
ahmail
korine
ceraldo
arlani
matleigh
lionna
anaziah
mikaimar
lyrice
saels
annal


6